# Análisis de el Retraso de las líneas aéreas de los Estados Unidos
- Se hace el análisis de grandes volúmenes de información usando herramientas especialmente diseñadas para el manejo de bases de datos de este calibre, se usará Spark mediante PySpark con Python para lograr este fin.

## Se prepara el entorno con PySpark para Google Colab

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.5.7/spark-3.5.7-bin-hadoop3.tgz
!tar -xzf spark-3.5.7-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 https://cli.github.com/packages stable InRelease
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.7-bin-hadoop3"

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark") \
    .master("local[*]") \
    .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/02 04:44:13 WARN Utils: Your hostname, nixos-desktop, resolves to a loopback address: 127.0.0.2; using 192.168.100.38 instead (on interface enp9s0)
25/12/02 04:44:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/02 04:44:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Se monta la unidad de Google Drive para poder obtener los CSVs sobre los que PySpark actuará para el análisis de datos que se va a aplicar.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ln -s /content/drive/MyDrive/ColabShared/airline_delay_analysis/ /content/csvs
!ls /content/csvs/

2009.csv  2011.csv  2013.csv  2015.csv	2017.csv  2019.csv_  processed
2010.csv  2012.csv  2014.csv  2016.csv	2018.csv  20.csv_


In [6]:
# Sección de rutas a usar, se mantienen como variables a conveniencia
ruta_general_csvs = "/content/csvs/"
ruta_general_processed = "/content/csvs/processed/"

## Se prepara el entorno con PySpark para instalación local

In [1]:
# Usar para una instalación local de Spark en Nix
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkLocal") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.debug.maxToStringFields", "100")\
    .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/02 14:55:11 WARN Utils: Your hostname, nixos-desktop, resolves to a loopback address: 127.0.0.2; using 192.168.100.38 instead (on interface enp9s0)
25/12/02 14:55:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/02 14:55:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Rutas para versión local
ruta_general_csvs_local = "csv/"
ruta_salida_csvs_local = "csvs_processed/"
ruta_salida_csvs_procesados = "csvs_cleaned/"

## Sección de funciones a usar en el código

In [5]:
# Funciones para guardado (volcado en almacenamiento) de CSVs
def guardar_csvs_en_almacenamiento(df_a_guardar, ruta_guardado:str):
    df_a_guardar.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(ruta_guardado)

# Función para cargar datos del almacenamiento a la RAM
def cargar_csvs(ruta_a_leer):
    df = spark.read.option("header", "true").csv(ruta_a_leer)
    return df

## Preprocesamiento de datos

Como los CSVs que se tienen poseen estructuras un tanto diferentes a partir del 2018, se buscará formar un CSV nuevo a partir de todos para poder estandarizar su estructura.

Se tomará en cuenta únicamente lo de 10 años que abarca de 2009 a 2018

In [9]:
# Importación de funciones/utilidades de pyspark.sql a usar
from pyspark.sql.functions import col, when, count, isnan, date_format, lit, concat

In [7]:
# Juntar todos los DataFrames (del 2009 al 2018) en uno solo para su análisis
df_csvs = spark.read.option("header", "True").option("inferSchema", "True").csv(ruta_general_csvs_local)
total_instancias = df_csvs.count()
print(f"Total de instancias: {total_instancias}")
df_csvs

Total de instancias: 61556964


FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
2009-01-01,XE,1204,DCA,EWR,1100.0,1058.0,-2.0,18.0,1116.0,1158.0,8.0,1202.0,1206.0,4.0,0.0,NULL,0.0,62.0,68.0,42.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1206,EWR,IAD,1510.0,1509.0,-1.0,28.0,1537.0,1620.0,4.0,1632.0,1624.0,-8.0,0.0,NULL,0.0,82.0,75.0,43.0,213.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1207,EWR,DCA,1100.0,1059.0,-1.0,20.0,1119.0,1155.0,6.0,1210.0,1201.0,-9.0,0.0,NULL,0.0,70.0,62.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1208,DCA,EWR,1240.0,1249.0,9.0,10.0,1259.0,1336.0,9.0,1357.0,1345.0,-12.0,0.0,NULL,0.0,77.0,56.0,37.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1209,IAD,EWR,1715.0,1705.0,-10.0,24.0,1729.0,1809.0,13.0,1900.0,1822.0,-38.0,0.0,NULL,0.0,105.0,77.0,40.0,213.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,ATL,EWR,1915.0,1913.0,-2.0,19.0,1932.0,2108.0,15.0,2142.0,2123.0,-19.0,0.0,NULL,0.0,147.0,130.0,96.0,745.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,CLE,ATL,1645.0,1637.0,-8.0,12.0,1649.0,1820.0,5.0,1842.0,1825.0,-17.0,0.0,NULL,0.0,117.0,108.0,91.0,554.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1214,DCA,EWR,1915.0,1908.0,-7.0,9.0,1917.0,1953.0,34.0,2035.0,2027.0,-8.0,0.0,NULL,0.0,80.0,79.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1215,EWR,DCA,1715.0,1710.0,-5.0,28.0,1738.0,1819.0,4.0,1838.0,1823.0,-15.0,0.0,NULL,0.0,83.0,73.0,41.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1217,EWR,DCA,1300.0,1255.0,-5.0,15.0,1310.0,1349.0,7.0,1408.0,1356.0,-12.0,0.0,NULL,0.0,68.0,61.0,39.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL


In [9]:
# Eliminar columna innecesaria que viene por defecto en cada CSV
df_original = df_csvs.drop('Unnamed: 27')
# Guardar los CSV al almacenamiento sin la columna innecesaria
guardar_csvs_en_almacenamiento(df_original, ruta_salida_csvs_local)
df_original

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2009-01-01,XE,1204,DCA,EWR,1100.0,1058.0,-2.0,18.0,1116.0,1158.0,8.0,1202.0,1206.0,4.0,0.0,NULL,0.0,62.0,68.0,42.0,199.0,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1206,EWR,IAD,1510.0,1509.0,-1.0,28.0,1537.0,1620.0,4.0,1632.0,1624.0,-8.0,0.0,NULL,0.0,82.0,75.0,43.0,213.0,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1207,EWR,DCA,1100.0,1059.0,-1.0,20.0,1119.0,1155.0,6.0,1210.0,1201.0,-9.0,0.0,NULL,0.0,70.0,62.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1208,DCA,EWR,1240.0,1249.0,9.0,10.0,1259.0,1336.0,9.0,1357.0,1345.0,-12.0,0.0,NULL,0.0,77.0,56.0,37.0,199.0,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1209,IAD,EWR,1715.0,1705.0,-10.0,24.0,1729.0,1809.0,13.0,1900.0,1822.0,-38.0,0.0,NULL,0.0,105.0,77.0,40.0,213.0,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,ATL,EWR,1915.0,1913.0,-2.0,19.0,1932.0,2108.0,15.0,2142.0,2123.0,-19.0,0.0,NULL,0.0,147.0,130.0,96.0,745.0,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,CLE,ATL,1645.0,1637.0,-8.0,12.0,1649.0,1820.0,5.0,1842.0,1825.0,-17.0,0.0,NULL,0.0,117.0,108.0,91.0,554.0,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1214,DCA,EWR,1915.0,1908.0,-7.0,9.0,1917.0,1953.0,34.0,2035.0,2027.0,-8.0,0.0,NULL,0.0,80.0,79.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1215,EWR,DCA,1715.0,1710.0,-5.0,28.0,1738.0,1819.0,4.0,1838.0,1823.0,-15.0,0.0,NULL,0.0,83.0,73.0,41.0,199.0,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1217,EWR,DCA,1300.0,1255.0,-5.0,15.0,1310.0,1349.0,7.0,1408.0,1356.0,-12.0,0.0,NULL,0.0,68.0,61.0,39.0,199.0,NULL,NULL,NULL,NULL,NULL


In [10]:
# Determinar la cantidad de instancias que no fueron canceladas
df_canceladas_filtered = df_original.filter(col("CANCELLED") == 1).count()
print(f"Total de vuelos cancelados: {df_canceladas_filtered}")

[Stage 13:=================================================>      (51 + 7) / 58]

Total de vuelos cancelados: 973209


### Se preparan los datos dejando las columnas a usar para formar los conjuntos que se usarán en los modelos

In [11]:
# Lista de atributos a usar para el objetivo
atributos_utiles = [
    'FL_DATE', 'OP_CARRIER', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'DISTANCE'
]

In [12]:
# Obtener las instancias útiles (no canceladas)
df_sin_canceladas = df_original.filter(col("CANCELLED") == 0)
# Como la variable importante para el objetivo es 'DEP_DELAY' y con eso se determina si un vuelo está a tiempo o no se crea nueva columna
# Si es mayor a 15 minutos se considerará como retraso en caso contrario como 'a tiempo', convirtiéndola en clasificación binaria
df_late = df_sin_canceladas.withColumn("LATE", when(col("DEP_DELAY") > 15, 1.0).otherwise(0.0))
df_late

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,LATE
2009-01-01,XE,1204,DCA,EWR,1100.0,1058.0,-2.0,18.0,1116.0,1158.0,8.0,1202.0,1206.0,4.0,0.0,NULL,0.0,62.0,68.0,42.0,199.0,NULL,NULL,NULL,NULL,NULL,0.0
2009-01-01,XE,1206,EWR,IAD,1510.0,1509.0,-1.0,28.0,1537.0,1620.0,4.0,1632.0,1624.0,-8.0,0.0,NULL,0.0,82.0,75.0,43.0,213.0,NULL,NULL,NULL,NULL,NULL,0.0
2009-01-01,XE,1207,EWR,DCA,1100.0,1059.0,-1.0,20.0,1119.0,1155.0,6.0,1210.0,1201.0,-9.0,0.0,NULL,0.0,70.0,62.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,0.0
2009-01-01,XE,1208,DCA,EWR,1240.0,1249.0,9.0,10.0,1259.0,1336.0,9.0,1357.0,1345.0,-12.0,0.0,NULL,0.0,77.0,56.0,37.0,199.0,NULL,NULL,NULL,NULL,NULL,0.0
2009-01-01,XE,1209,IAD,EWR,1715.0,1705.0,-10.0,24.0,1729.0,1809.0,13.0,1900.0,1822.0,-38.0,0.0,NULL,0.0,105.0,77.0,40.0,213.0,NULL,NULL,NULL,NULL,NULL,0.0
2009-01-01,XE,1212,ATL,EWR,1915.0,1913.0,-2.0,19.0,1932.0,2108.0,15.0,2142.0,2123.0,-19.0,0.0,NULL,0.0,147.0,130.0,96.0,745.0,NULL,NULL,NULL,NULL,NULL,0.0
2009-01-01,XE,1212,CLE,ATL,1645.0,1637.0,-8.0,12.0,1649.0,1820.0,5.0,1842.0,1825.0,-17.0,0.0,NULL,0.0,117.0,108.0,91.0,554.0,NULL,NULL,NULL,NULL,NULL,0.0
2009-01-01,XE,1214,DCA,EWR,1915.0,1908.0,-7.0,9.0,1917.0,1953.0,34.0,2035.0,2027.0,-8.0,0.0,NULL,0.0,80.0,79.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,0.0
2009-01-01,XE,1215,EWR,DCA,1715.0,1710.0,-5.0,28.0,1738.0,1819.0,4.0,1838.0,1823.0,-15.0,0.0,NULL,0.0,83.0,73.0,41.0,199.0,NULL,NULL,NULL,NULL,NULL,0.0
2009-01-01,XE,1217,EWR,DCA,1300.0,1255.0,-5.0,15.0,1310.0,1349.0,7.0,1408.0,1356.0,-12.0,0.0,NULL,0.0,68.0,61.0,39.0,199.0,NULL,NULL,NULL,NULL,NULL,0.0


In [13]:
# Formar el DataFrame con las columnas necesarias más variable objetivo
df_ajustado = df_late.select(*atributos_utiles, "LATE")
df_ajustado

FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,LATE
2009-01-01,XE,DCA,EWR,1100.0,1202.0,62.0,199.0,0.0
2009-01-01,XE,EWR,IAD,1510.0,1632.0,82.0,213.0,0.0
2009-01-01,XE,EWR,DCA,1100.0,1210.0,70.0,199.0,0.0
2009-01-01,XE,DCA,EWR,1240.0,1357.0,77.0,199.0,0.0
2009-01-01,XE,IAD,EWR,1715.0,1900.0,105.0,213.0,0.0
2009-01-01,XE,ATL,EWR,1915.0,2142.0,147.0,745.0,0.0
2009-01-01,XE,CLE,ATL,1645.0,1842.0,117.0,554.0,0.0
2009-01-01,XE,DCA,EWR,1915.0,2035.0,80.0,199.0,0.0
2009-01-01,XE,EWR,DCA,1715.0,1838.0,83.0,199.0,0.0
2009-01-01,XE,EWR,DCA,1300.0,1408.0,68.0,199.0,0.0


In [14]:
print("Total de instacias: ", df_ajustado.count())

[Stage 20:=====================================================>  (55 + 3) / 58]

Total de instacias:  60583755


In [18]:
# 1. Definimos las columnas que quieres revisar (tus atributos importantes)
# Si ya tienes el DF filtrado con select, puedes usar 'df.columns'
columnas_importantes = df_ajustado.columns

# 2. Construimos la "Query Maestra" usando una List Comprehension
# Lógica: "Para cada columna 'c', cuenta 1 si es Nulo, de lo contrario nada."
expresiones_nulos = [count(when(col(c).isNull(), c)).alias(c) for c in columnas_importantes]

# 3. Ejecutamos el cálculo (Spark lo hace en paralelo)
resultado_nulos = df_ajustado.select(*expresiones_nulos)

# 4. Mostramos el resultado
# vertical=True es vital aquí para leerlo bien lista hacia abajo
resultado_nulos.show(vertical=True)

[Stage 24:==================================================>     (52 + 6) / 58]

-RECORD 0---------------
 FL_DATE          | 0   
 OP_CARRIER       | 0   
 ORIGIN           | 0   
 DEST             | 0   
 CRS_DEP_TIME     | 1   
 CRS_ARR_TIME     | 2   
 CRS_ELAPSED_TIME | 23  
 DISTANCE         | 0   
 LATE             | 0   



In [20]:
# Eliminar registros nulos
df_para_modelo = df_ajustado.dropna()
print("Total de instacias: ", df_para_modelo.count())

[Stage 27:==================================================>     (52 + 6) / 58]

Total de instacias:  60583731


In [22]:
# Guardar el DF con menos atributos y menos instancias
guardar_csvs_en_almacenamiento(df_para_modelo, ruta_salida_csvs_procesados)

ConnectionRefusedError: [Errno 111] Connection refused

In [6]:
# Cargar los CSV a un DataFrame de Spark
df_cargado = cargar_csvs(ruta_salida_csvs_procesados)
df_cargado.show(5)

+----------+----------+------+----+------------+------------+----------------+--------+----+
|   FL_DATE|OP_CARRIER|ORIGIN|DEST|CRS_DEP_TIME|CRS_ARR_TIME|CRS_ELAPSED_TIME|DISTANCE|LATE|
+----------+----------+------+----+------------+------------+----------------+--------+----+
|2009-08-31|        9E|   ORF| ATL|      1159.0|      1400.0|           121.0|   516.0| 0.0|
|2009-08-31|        9E|   ATL| PIT|       725.0|       910.0|           105.0|   526.0| 0.0|
|2009-08-31|        9E|   PIT| ATL|       941.0|      1129.0|           108.0|   526.0| 0.0|
|2009-08-31|        9E|   ATL| MCI|       915.0|      1037.0|           142.0|   692.0| 0.0|
|2009-08-31|        9E|   TYS| ATL|       540.0|       638.0|            58.0|   152.0| 0.0|
+----------+----------+------+----+------------+------------+----------------+--------+----+
only showing top 5 rows


El proceso de caracterización de la población se realizará en dos fases, de forma general se hará tomando en cuenta el tiempo (las estaciones del año para ser más específico), con los ya conocidos espacios temporales de 3 meses cada uno.

La segunda fase será de acuerdo al porcentaje de distribución de los pasajeros de acuerdo al listado de los 30 aeropuertos más usados de los Estados Unidos también conocido como el 'Core 30' que acumulan aproximadamente el 77% de los viajes aeronaúticos.

Por lo que se harán dos grupos grandes, el mayoritario de 77% corresponderá al grupo del Core30, mientras que los demás (23%) se clasificarán como 'NoCore', así no se excluirán aeropuertos que dispongan de menos viajes.

In [7]:
# Porcentajes de vuelo por estación del año
porcentajes = {
    'primavera': 0.275,
    'verano': 0.258,
    'otono': 0.249,
    'invierno': 0.218
}

# Meses del año por estación
meses_primavera = ['03', '04', '05']
meses_verano = ['06', '07', '08']
meses_otono = ['09', '10', '11']
meses_invierno = ['12', '01', '02']

# Listado de los aeropuertos más usados de los Estados Unidos, los que no se encuentren en esta lista
# se considerarán parte de un grupo complementario.
core_30 = [
    'ATL', 'DFW', 'DEN', 'LAX', 'ORD',
    'JFK', 'MCO', 'LAS', 'CLT', 'MIA',
    'SEA', 'EWR', 'SFO', 'PHX', 'IAH',
    'BOS', 'FLL', 'MSP', 'LGA', 'DTW',
    'PHL', 'SLC', 'BWI', 'DCA', 'SAN',
    'IAD', 'TPA', 'MDW', 'HNL', 'MEM'
]
pct_core30 = 0.77
pct_otros = 0.23

In [10]:
df_final = df_cargado

# Crear columna del mes (texto 'MM')
df_prep = df_final.withColumn("Month_Str", date_format(col("FL_DATE"), "MM"))

# Crear Etiqueta de Estación (Stage 1)
df_stage1 = df_prep.withColumn("SEASON",
    when(col("Month_Str").isin(meses_invierno), "Invierno")
    .when(col("Month_Str").isin(meses_primavera), "Primavera")
    .when(col("Month_Str").isin(meses_verano), "Verano")
    .when(col("Month_Str").isin(meses_otono), "Otono")
)

# Crear Etiqueta de Aeropuerto (Stage 2)
# Si está en la lista -> 'Core30', si no -> 'Others'
df_stage2 = df_stage1.withColumn("AIRPORT_TYPE",
    when(col("ORIGIN").isin(core_30), "Core30")
    .otherwise("Others")
)

# CREAR LA SUPER-LLAVE (Concatenación)
# Ejemplo de resultado: "Primavera_Core30", "Invierno_Others"
df_strata = df_stage2.withColumn("STRATA", 
    concat(col("SEASON"), lit("_"), col("AIRPORT_TYPE"))
)

df_strata.show(5)

+----------+----------+------+----+------------+------------+----------------+--------+----+---------+------+
|   FL_DATE|OP_CARRIER|ORIGIN|DEST|CRS_DEP_TIME|CRS_ARR_TIME|CRS_ELAPSED_TIME|DISTANCE|LATE|Month_Str|SEASON|
+----------+----------+------+----+------------+------------+----------------+--------+----+---------+------+
|2009-08-31|        9E|   ORF| ATL|      1159.0|      1400.0|           121.0|   516.0| 0.0|       08|Verano|
|2009-08-31|        9E|   ATL| PIT|       725.0|       910.0|           105.0|   526.0| 0.0|       08|Verano|
|2009-08-31|        9E|   PIT| ATL|       941.0|      1129.0|           108.0|   526.0| 0.0|       08|Verano|
|2009-08-31|        9E|   ATL| MCI|       915.0|      1037.0|           142.0|   692.0| 0.0|       08|Verano|
|2009-08-31|        9E|   TYS| ATL|       540.0|       638.0|            58.0|   152.0| 0.0|       08|Verano|
+----------+----------+------+----+------------+------------+----------------+--------+----+---------+------+
only showi

In [11]:
print("Total instancias: ", df_strata.count())

[Stage 6:===============================================>         (24 + 5) / 29]

Total instancias:  60583731


# Las celdas de aquí en adelante fueron para hacer pruebas de comandos de PySpark para poder averigüar que método seguir para hacer un muestreo de acuerdo a la caracterización

In [10]:
#ruta_salida = "/content/csvs/processed"

# Guardar
# mode("overwrite"): Si la carpeta ya existe, la borra y la crea de nuevo.
df_truncated.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(ruta_salida_csvs_local)

In [ ]:
# Para probar, se carga un solo CSV como DataFrame de Spark
df_prueba = spark.read.option("header", "True").option("inferSchema", "True").csv('/content/csvs/2009.csv')

In [ ]:
df_prueba.show(5)

In [ ]:
from pyspark.sql.functions import when, col, date_format

# Usamos date_format para asegurar que salga '01' y no el número 1
df_prep = df_prueba.withColumn("Month_Str", date_format(col("FL_DATE"), "MM"))

df_final = df_prep.withColumn("SEASON",
    when(col("Month_Str").isin(meses_invierno), "Invierno")
    .when(col("Month_Str").isin(meses_primavera), "Primavera")
    .when(col("Month_Str").isin(meses_verano), "Verano")
    .when(col("Month_Str").isin(meses_otono), "Otono")
    .otherwise("Unknown") # Por seguridad, aunque no debería ocurrir
).drop("Month_Str") # Borramos la columna auxiliar

# 4. Verificamos
df_final.select("FL_DATE", "SEASON").show(5)